## Historical Load : a subset of 150Gb of trips data

In [2]:
from pyspark.sql.functions import col,lit
import pyspark.sql.functions as func


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# normalisation des colonnes 

columns_names = "vendor_id,pickup_datetime,dropoff_datetime,passenger_count," \
    + "trip_distance,rate_code,store_and_fwd_flag,pickup_longitude,pickup_latitude," \
    + "dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount," \
    + "tolls_amount,improvement_surcharge,total_amount"

columns_names = columns_names.split(",")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
subset_table_path = "s3://datalake-vtc/raw_data/gcs/tripdatatest/data/2023-10-06**.parquet"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
parquet_table.describe().show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------------------------------------------------------------+-----------------+-------------------+------------------+-----------------+------------------+--------------------+------------------+-----------------+------------------+-------------------+-------------------+------------------+-------------------+---------------------+-----------------+--------------------+-----------+------------------+------------------+---------+-----------------+-----------------+------------------+-------------------+
|summary|_file                                                                  |_line            |vendor_id          |passenger_count   |trip_distance    |ratecode_id       |store_and_fwd_flag  |pulocation_id     |dolocation_id    |fare_amount       |extra              |mta_tax            |tip_amount        |tolls_amount       |improvement_surcharge|total_amount     |congestion_surcharge|airport_fee|pickup_longitude  |pickup_latitude   |rate_code|dropoff_longitude|dr

In [ ]:
parquet_table = spark.read.format("parquet").option('mergeSchema','true').load(subset_table_path)

### Loading files 2010 -> 2014 ...

In [ ]:
data_v1 = parquet_table.filter(col("_file").rlike("yellow_tripdata_201[0-4]-..\\.parquet"))

data_v1 = data_v1 \
    .withColumn('improvement_surcharge', func.lit(0)) \
    .withColumn('extra', col('surcharge'))

final_data = data_v1.select(columns_names)

### Loading files 2015-01 -> 2015-06 ...

In [ ]:
data_v2 = parquet_table.filter(col("_file").rlike("yellow_tripdata_2015-0[1-6]\\.parquet"))

data_v2 = data_v2.select(columns_names)

final_data = final_data.union(data_v2)


## Loading files 2015-07 -> 2016-06 ...

In [ ]:
data_v3 = parquet_table.filter(col("_file").rlike("yellow_tripdata_2015-(07|08|09|10|11|12)\\.parquet"))

data_v3 = data_v3.union(
    parquet_table.filter(col("_file")
                         .rlike("yellow_tripdata_2016-(01|02|03|04|05|06)\\.parquet")))

data_v3 = data_v3.select(columns_names)
    
final_data = final_data.union(data_v3)
    
final_data = final_data \
    .withColumn('pu_location_id', func.lit(None)) \
    .withColumn('do_location_id', func.lit(None)) \
    .withColumn('with_areas', func.lit(False))    

In [ ]:
new_columns_names = columns_names + ['pu_location_id', 'do_Location_id', 'with_areas']

## Loading files 2016-07 -> 2018 ...

In [ ]:
data_v4 = parquet_table.filter(col("_file").rlike("yellow_tripdata_2016-(07|08|09|10|11|12)\\.parquet"))

data_v4 = data_v4.union(
    parquet_table.filter(col("_file")
                         .rlike("yellow_tripdata_201[7-8]-..\\.parquet")))
data_v4.columns

In [ ]:
new_columns_names = columns_names + ['pu_location_id', 'do_location_id', 'with_areas']

data_v4 = data_v4 \
    .withColumn('with_areas', func.lit(True)) \
    .withColumn('pickup_datetime',col('tpep_pickup_datetime')) \
    .withColumn('dropoff_datetime',col('tpep_dropoff_datetime')) \
    .withColumn('rate_code', col('ratecode_id')) \
    .withColumnRenamed('pulocation_id','pu_location_id')\
    .withColumnRenamed('dolocation_id','do_location_id')\
    .select(new_columns_names)
                

In [ ]:
data_v4.printSchema()

In [ ]:
final_data = final_data.union(data_v4)

## Cast columns 

In [ ]:
final_data = final_data \
    .withColumn("pickup_datetime", func.to_timestamp("pickup_datetime", 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn("dropoff_datetime", func.to_timestamp("dropoff_datetime", 'yyyy-MM-dd HH:mm:ss'))

In [ ]:
final_data = final_data \
    .withColumn("passenger_count", final_data.passenger_count.cast("int")) \
    .withColumn("trip_distance", final_data.trip_distance.cast("float")) \
    .withColumn("payment_type", final_data.payment_type.cast("float")) \
    .withColumn("pickup_longitude", final_data.pickup_longitude.cast("float")) \
    .withColumn("pickup_latitude", final_data.pickup_latitude.cast("float")) \
    .withColumn("dropoff_longitude", final_data.dropoff_longitude.cast("float")) \
    .withColumn("dropoff_latitude", final_data.dropoff_latitude.cast("float")) \
    .withColumn("fare_amount", final_data.fare_amount.cast("float")) \
    .withColumn("extra", final_data.extra.cast("float")) \
    .withColumn("mta_tax", final_data.mta_tax.cast("float")) \
    .withColumn("tip_amount", final_data.tip_amount.cast("float")) \
    .withColumn("tolls_amount", final_data.tolls_amount.cast("float")) \
    .withColumn("improvement_surcharge", final_data.improvement_surcharge.cast("float")) \
    .withColumn("total_amount", final_data.total_amount.cast("float")) \
    .withColumn("pu_location_id", final_data.pu_location_id.cast("int")) \
    .withColumn("do_location_id", final_data.do_location_id.cast("int"))

In [ ]:
final_data.sample(False, 0.1)

In [ ]:
dbname = "dev"
host = "jdbc:redshift:iam://dwh-vtc-trips.371858328372.eu-west-3.redshift-serverless.amazonaws.com/dev"
port = "5439"

s3_path = "s3://datalake-vtc/raw_data/gcs/tripdatatest/"
iam_role_arn = "arn:aws:iam::371858328372:role/service-role/AmazonRedshift-CommandsAccessRole-20230929T161605"



final_data.sample(False, 0.1).write \
    .format("io.github.spark_redshift_community.spark.redshift") \
    .option("url", host) \
    .option("dbtable", "gsc.trips") \
    .option("tempdir", "s3://datalake-vtc/tmp_redshift/") \
    .option("aws_iam_role", iam_role_arn) \
    .mode("append") \
    .save()